In [ ]:
import os
import re
from configparser import ConfigParser
from boto3        import session

Variable Collection

In [ ]:
parser = ConfigParser()
parser.read('config.cfg')

Connection Configuration

In [ ]:
conn = session.Session()
client = conn.client(
    's3',
    region_name=parser.get('aws_digital_ocean', 'REGION_NAME'),
    endpoint_url=parser.get('aws_digital_ocean', 'ENDPOINT'),
    aws_access_key_id=parser.get('aws_digital_ocean', 'ACCESS_ID'),
    aws_secret_access_key=parser.get('aws_digital_ocean', 'SECRET_KEY')
)

Pagination Activation

In [ ]:
paginator = client.get_paginator('list_objects_v2')

List Prefix

In [ ]:
list_pages = []
for prefix in parser['prefix_buckets']:
    bucket = parser.get('aws_digital_ocean', 'BUCKET')
    prefix_const = f"{parser.get('prefix_buckets', prefix)}"

    pages = paginator.paginate(Bucket=bucket, Prefix=prefix_const)

    for page in pages:
        if page['KeyCount'] > 0:
            for obj in page['Contents']:
                if len(re.findall(r'.jsonl', obj['Key']))>0:
                    list_pages.append(
                        {
                            'path_download': prefix,
                            'file_name': obj['Key']
                        }
                    )

In [ ]:
print(len(list_pages))

Download Files

In [ ]:
for item in list_pages:
    FILE_NAME = item['file_name'].split('/')[-1]          
    PATH_DOWNLOAD = parser.get('paths', item['path_download'])

    if not os.path.exists(PATH_DOWNLOAD):
        os.makedirs(PATH_DOWNLOAD)

    FILE_DONWLOAD = f"{PATH_DOWNLOAD}/{FILE_NAME}"   
    print(FILE_NAME, PATH_DOWNLOAD, FILE_DONWLOAD)     
    with open(FILE_DONWLOAD, 'wb') as f:
        client.download_fileobj(parser.get('aws_digital_ocean', 'BUCKET'), item['file_name'], f)